In [11]:
import numpy as np
import plotly.graph_objects as go

# === Paramètres ===
Te = 1.0
N = 100

F = np.array([
    [1, 0, Te,  0],
    [0, 1,  0, Te],
    [0, 0,  1,  0],
    [0, 0,  0,  1]
])

H = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0]
])

Q =  np.eye(4)
R = 30 * np.eye(2)

x_true = np.array([0, 0, 1, 1])
x_est = np.array([0, 0, 0, 0])
P = 1000* np.eye(4)

entropy_hist = []

def shannon_entropy_gaussian(P):
    n = P.shape[0]
    sign, logdet = np.linalg.slogdet(P)
    if sign <= 0:
        return np.nan
    return 0.5 * (n * np.log(2 * np.pi * np.e) + logdet)

# === Boucle Kalman ===
for _ in range(N):
    w = np.random.multivariate_normal(np.zeros(4), Q)
    x_true = F @ x_true + w

    v = np.random.multivariate_normal(np.zeros(2), R)
    z = H @ x_true + v

    x_pred = F @ x_est
    P_pred = F @ P @ F.T + Q

    y = z - H @ x_pred
    S = H @ P_pred @ H.T + R
    K = P_pred @ H.T @ np.linalg.inv(S)
    x_est = x_pred + K @ y
    P = (np.eye(4) - K @ H) @ P_pred

    entropy_hist.append(shannon_entropy_gaussian(P))

# === Affichage interactif avec Plotly ===
fig_entropy = go.Figure()
fig_entropy.add_trace(go.Scatter(
    x=list(range(N)),
    y=entropy_hist,
    mode='lines',
    name='Entropie'
))
fig_entropy.update_layout(
    title="Évolution de l'entropie de Shannon H(x)",
    xaxis_title="Temps",
    yaxis_title="Entropie",
    
)
fig_entropy.show()
